### 物体图像分类
大部分代码均沿用task4 此任务主要改变dropout进行对比

以下用自己的理解陈述一下过拟合的一般解决方案：

    1）增加数据集
    2）降低训练复杂度
    
降低复杂度常用的使用方法：
    
    1）正则化
    2）dropout
    3）其他降低模型复杂度的参数（如机器学习中树的深度等）

此处为深度学习，主要讨论前两种

1. 正则化：L1正则化，即在原有的损失函数的基础上添加参数向量的L1范数。可以简单理解为对系数绝对值之和加入惩罚项以达到减少过拟合的情况。L1正则化可以使系数降为0。
               L2正则化，即在原有的损失函数的基础上添加参数向量的L2范数。可以简单理解为系数平方和加入惩罚项以达到减少过拟合的情况。L2正则化使系数趋近为0。
               
               pytorch自带的损失函数只有L2正则化参数（weight_decay）,L1正则化则需要手动实现。
               
               
               
2. dropout：dropout主要是让神经元的激活值以一定的概率停止工作，从而降低复杂度是模型泛化性更强。

注：为了加快运算速度，下面的训练调参较为简单，主要为了方便对比正则化结果


#### 加载数据集

In [1]:
import torchvision as tv
import torch as t
import torchvision.transforms as transforms 
from torchvision.transforms import ToPILImage
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [2]:
#对数据预处理
trans = transforms.Compose([
    transforms.ToTensor(), #转为tensor
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

#下载数据
#训练数据
trainset = tv.datasets.CIFAR10('./',train=True,download=True,transform = trans)
trainloader = t.utils.data.DataLoader(trainset, 
                                                    batch_size = 4,
                                                    shuffle = True,
                                                    num_workers =2) 
#测试数据
testset = tv.datasets.CIFAR10('./',train=False,download=True,transform = trans)
testloader = t.utils.data.DataLoader(testset, 
                                                    batch_size = 4,
                                                    shuffle = True,
                                                    num_workers =2) 

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.conv1 = nn.Conv2d(3,16,3,padding=1) #输入通道数，输出通道数，卷积层
        self.conv2 = nn.Conv2d(16,32,3,padding=1)
        self.conv3 = nn.Conv2d(32,64,3,padding=1)
        self.pool = nn.MaxPool2d(2,2)  #池化层
        self.linear1 = nn.Linear(1024,512)   #64*2*2
        self.linear2 = nn.Linear(512,10)
    
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))  #卷积-激活函数-池化
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1,1024)  #reshape
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x

model = Model()

In [4]:
#定义损失函数和优化器
import torch.optim as optim

loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, weight_decay = 0, momentum = 0.9,  nesterov= True)  
#lr: 学习率
#weight_decay：基本思想就是减小不重要的参数对最后结果的影响，网络中有用的权重则不会收到Weight decay影响
#nesterov: 确定是否使用Nesterov动量
#moment: 动量

In [5]:
for epoch in range(2):
    train_loss, test_loss = [], []
    #训练
    model.train()
    for i,data in enumerate(trainloader,0):
        data,y = data
        optimizer.zero_grad()  #梯度清零
        output = model(data)
        loss = loss_func(output,y)
        loss.backward()
        #更新参数
        optimizer.step()
        train_loss.append(loss.item())
        if i%2000 == 1999:
            print('[%d,%5d] loss: %.3f' %(epoch+1,i+1,train_loss[i]))
    #评价
    model.eval()
    for data,y in testloader:
        output = model(data)
        loss = loss_func(output,y)
        test_loss.append(loss.item())

[1, 2000] loss: 1.632
[1, 4000] loss: 1.727
[1, 6000] loss: 1.465
[1, 8000] loss: 2.716
[1,10000] loss: 2.293
[1,12000] loss: 1.965
[2, 2000] loss: 1.475
[2, 4000] loss: 1.842
[2, 6000] loss: 1.732
[2, 8000] loss: 2.270
[2,10000] loss: 1.997
[2,12000] loss: 2.626


In [6]:
#测试集表现
dataiter = iter(testloader) 
data,labels = dataiter.next()  #每个批次4个样本
output = model(data)
_,preds = t.max(output,1)  #axis=1

In [22]:
#np.squeeze(labels.numpy())

array([0, 8, 9, 1])

In [33]:
labels[0]

tensor(5)

In [7]:
print("actual:", [classes[labels[i]] for i in range(4)])
print("preds:" ,[classes[preds[i]] for i in range(4)])

actual: ['dog', 'cat', 'plane', 'car']
preds: ['deer', 'frog', 'car', 'car']


In [8]:
correct = 0 
total = 0

for data,y in testloader:
    output = model(data)
    _, preds = t.max(output,1)
    total += y.size(0)
    correct += (preds == y).sum()
        
print("%d 张测试图片准确率为：%d %%" %(total,(100*correct/total)))


## 为了节省计算资源 只迭代了2次 以致准确率较低，实际项目中要增加迭代次数

10000 张测试图片准确率为：36 %


### dropout

In [9]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.conv1 = nn.Conv2d(3,16,3,padding=1) #输入通道数，输出通道数，卷积层
        self.conv2 = nn.Conv2d(16,32,3,padding=1)
        self.conv3 = nn.Conv2d(32,64,3,padding=1)
        self.pool = nn.MaxPool2d(2,2)  #池化层
        self.dropout = nn.Dropout(0.5)
        self.linear1 = nn.Linear(1024,512)   #64*2*2
        self.linear2 = nn.Linear(512,10)
    
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))  #卷积-激活函数-池化
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1,1024)  #reshape
        x = F.relu(self.linear1(x))
        x = self.dropout (x)
        x = self.linear2(x)
        return x

model = Model()

In [10]:
import torch.optim as optim

#正则化dropout=0.5 L2=0
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, weight_decay = 0, momentum = 0.9,  nesterov= True)  

In [11]:
for epoch in range(2):
    train_loss, test_loss = [], []
    #训练
    model.train()
    for i,data in enumerate(trainloader,0):
        data,y = data
        optimizer.zero_grad()  #梯度清零
        output = model(data)
        loss = loss_func(output,y)
        loss.backward()
        #更新参数
        optimizer.step()
        train_loss.append(loss.item())
        if i%2000 == 1999:
            print('[%d,%5d] loss: %.3f' %(epoch+1,i+1,train_loss[i]))
    #评价
    model.eval()
    for data,y in testloader:
        output = model(data)
        loss = loss_func(output,y)
        test_loss.append(loss.item())

[1, 2000] loss: 2.449
[1, 4000] loss: 2.107
[1, 6000] loss: 2.345
[1, 8000] loss: 2.103
[1,10000] loss: 2.082
[1,12000] loss: 2.715
[2, 2000] loss: 1.451
[2, 4000] loss: 1.612
[2, 6000] loss: 2.237
[2, 8000] loss: 1.857
[2,10000] loss: 2.252
[2,12000] loss: 2.346


In [21]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.conv1 = nn.Conv2d(3,16,3,padding=1) #输入通道数，输出通道数，卷积层
        self.conv2 = nn.Conv2d(16,32,3,padding=1)
        self.conv3 = nn.Conv2d(32,64,3,padding=1)
        self.pool = nn.MaxPool2d(2,2)  #池化层
        self.linear1 = nn.Linear(1024,512)   #64*2*2
        self.linear2 = nn.Linear(512,10)
    
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))  #卷积-激活函数-池化
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1,1024)  #reshape
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x

model = Model()

In [22]:
#正则化L2=0.01
optimizer = optim.SGD(model.parameters(), lr = 0.01, weight_decay = 1, momentum = 0.9,  nesterov= True)  

for epoch in range(2):
    train_loss, test_loss = [], []
    #训练
    model.train()
    for i,data in enumerate(trainloader,0):
        data,y = data
        optimizer.zero_grad()  #梯度清零
        output = model(data)
        loss = loss_func(output,y)
        loss.backward()
        #更新参数
        optimizer.step()
        train_loss.append(loss.item())
        if i%2000 == 1999:
            print('[%d,%5d] loss: %.3f' %(epoch+1,i+1,train_loss[i]))
    #评价
    model.eval()
    for data,y in testloader:
        output = model(data)
        loss = loss_func(output,y)
        test_loss.append(loss.item())

[1, 2000] loss: 2.309
[1, 4000] loss: 2.296
[1, 6000] loss: 2.288
[1, 8000] loss: 2.303
[1,10000] loss: 2.315
[1,12000] loss: 2.286
[2, 2000] loss: 2.296
[2, 4000] loss: 2.312
[2, 6000] loss: 2.314
[2, 8000] loss: 2.319
[2,10000] loss: 2.300
[2,12000] loss: 2.298


In [23]:
#测试集表现
dataiter = iter(testloader) 
data,labels = dataiter.next()  #每个批次4个样本
output = model(data)
_,preds = t.max(output,1)  #axis=1

In [24]:
print("actual:", [classes[labels[i]] for i in range(4)])
print("preds:" ,[classes[preds[i]] for i in range(4)])

actual: ['plane', 'horse', 'car', 'bird']
preds: ['truck', 'truck', 'truck', 'truck']


In [25]:
correct = 0 
total = 0

for data,y in testloader:
    output = model(data)
    _, preds = t.max(output,1)
    total += y.size(0)
    correct += (preds == y).sum()
        
print("%d 张测试图片准确率为：%d %%" %(total,(100*correct/total)))

##为了对比正则化的效果，正则化系数调整较大，可以看到在迭代次数过少的情况下严重欠拟合
#但在训练较为复杂时，适度正则化可以增加模型的鲁棒性

10000 张测试图片准确率为：10 %


可以看到，加入正则化之后，在模型训练次数较少时（上面例子迭代次数较少，为了节省计算时间，未增加迭代次数进行深入训练），模型更容易欠拟合；

另一方面，加入正则化之后，明显看到损失函数变化趋势波动减小，下降趋势变慢，更加平滑。